# GemPy Introduction

In this notebook, we look at modelling aspects with [GemPy](https://github.com/cgre-aachen/gempy) step-by-step. We'll only look at rather simple models, to be able to focus on the way GemPy works, and to get a feeling for the most important parameters for geological modeling with this software.

## Table of Contents

1. [Minimal needed inputs](#1)  
    1.1 [Minimal example](#1.1)
2. [How do `series` work?](#2)    
    2.1 [A model with 2 Layers in 1 Series](#2)  
    2.2 [A model mit 2 Layers in 2 Series](#2.2)  
    2.3 [Again a model with 2 layers in 2 Series ... with corrected data](#2.3)  
3. [Fault relations](#4)  
4. [Onlap relations](#5)  
    4.1 [_Inclined_  as Onlap](#5.1)  
    4.2 [_Inclined_ and _Flat_ as Onlap](#5.2)  
5. [Final thoughts](#6)  

In [ ]:
# Load in libraries
import numpy as np
import gempy as gp
%matplotlib inline

import theano  # type: ignore

theano.config.gcc.cxxflags = "-Wno-c++11-narrowing"

print("Run with gempy version {}".format(gp.__version__))

<a id='1'></a>
## Minimal needed inputs
GemPy needs two different data types as input data for models: *interface* and *orientation* data.

*Interface* data can be interpreted as points on a boundary between two model units. For instance, an interpreted seismic horizon, or the boundary between two geological units in a geological cross-section, or layer boundaries mapped in the field, ...

The format of _Interface_ points in a file (mostly as CSV files) looks like this:

| X  |  Y | Z |  formation |
|--:|--:|--:|---|
| 300  | 250  | 545  | sandstone  |
|  300 |  550 |  680 | limestone  |


<center>
    

<img src="https://upload.wikimedia.org/wikipedia/commons/8/80/Streichbild.svg" width=400 height=400>
   
</center>
Source: <a href="https://upload.wikimedia.org/wikipedia/commons/8/80/Streichbild.svg">Wikipedia</a>
<br>

*Orientation* data can be interpreted as the dip angle and strike of mapped layer boundaries. They control, how a modeled surface is oriented in space. While *Interface* data consist of X, Y, Z and the name of the unit, _Orientation_ data have additionally `azimuth` $\sigma$, `dip` $\varphi$, and `polarity`. 

The format of _Orientation_ data in a CSV file looks like this:

| X  |  Y |  Z |  azimuth | dip  | polarity  |  formation |
|--:|--:|--:|--:|--:|--:|---|
| 300  | 250  | 545  | 90  | 10  |  1 | sandstone  |
|  300 |  550 |  680 |  90 | 15  |  1 | limestone  |

<a id='1.1'></a>
### Minimal example
The absolute minimum to be parsed to GemPy are two *interface* points and one *orientation* point.

In [ ]:
# Create a gempy model
geo_model = gp.create_model('Simple_model')

In [ ]:
# Initial data:
# Which model object do we assign the data to (here geo_model)
# What is the model volume in m [xmin, xmax, ymin, ymax, zmin, zmax]
# What is the resolution of the regular grid [dx, dy, dz]
# optional: load in csv data

gp.init_data(geo_model, [0., 10., 0., 2., 0., 10.], [100, 3, 100],
            path_i = 'data/00_1Layer_simple_model_interfaces.csv',
            path_o = 'data/00_1Layer_simple_model_orientation.csv');

Once we generated the GemPy model object, and loaded in data, we can display the order of the units (like a stratigraphical column) of the model using `geo_model.surfaces`:

In [ ]:
# display the "stratigraphic column"
geo_model.surfaces

In [ ]:
# GemPy has some plot options
gp._plot.plot_data(geo_model)

GemPy is based on [Theano](https://de.wikipedia.org/wiki/Theano_(Deep_Learning)), a library for calculating and optimizing of mathematic expressions (especially multidimensional matrix-calculations). The developers of Theano stopped the project, which was picked up by the PyMC3 developers under the name [aesara](https://github.com/aesara-devs/aesara) and is actively developed under that name.

As GemPy uses a lot of matrix calculations (tensors), an efficient calculation is needed for making modeling feasibly on any hardware. The kriging parameters for the model are set up using `set_interpolator`.

For more information about Theano itself, I reccomend this [Tutorial](https://www.marekrei.com/blog/theano-tutorial/).

In [ ]:
# generate Theano Graph

gp.set_interpolator(geo_model,
                         compile_theano=True,
                         theano_optimizer='fast_compile',
                         verbose=[])

In [ ]:
# Compute the model
sol = gp.compute_model(geo_model, compute_mesh=False)

In [ ]:
gp._plot.plot_section(geo_model, cell_number=2, direction='y', show_data=True)

<a id='2'></a>
## How do Series work?
### A model with 2 layers and 1 Series
Based on the minimal example, we now add a second set of interface points to show what `Series` are in GemPy. Therefore, we have now two geological layers in our model (exkluding the _basement_):

In [ ]:
# Geo-model with 2 layers
geo_model = gp.create_model('2Layer_model')

In [ ]:
# initialize data
gp.init_data(geo_model, [0., 10., 0., 2., 0., 10.], [100, 3, 100],
            path_i = 'data/01_2Layer_simple_model_interfaces.csv',
            path_o = 'data/01_2Layer_simple_model_orientation.csv');

In [ ]:
geo_model.surfaces

In [ ]:
gp._plot.plot_data(geo_model)

In [ ]:
# Create the theano model
gp.set_interpolator(geo_model,
                         compile_theano=True,
                         theano_optimizer='fast_compile',
                         verbose=[])

In [ ]:
# Compute the model
sol = gp.compute_model(geo_model, compute_mesh=False)

In [ ]:
gp._plot.plot_section(geo_model, cell_number=2, direction='y', show_data=True)

This works only, because both layers are in the **same** Series, `default_series`.   
But what happens, if we define 2 series for the 2 units we have?

<a id='2.2'></a>
### A model with 2 layers in 2 Series

In [ ]:
gp.map_series_to_surfaces(geo_model,
                         {"Series_1": 'unit_1',
                         "Series_2": 'unit_2'})

In [ ]:
# Create the theano model
gp.set_interpolator(geo_model,
                         compile_theano=True,
                         theano_optimizer='fast_compile',
                         verbose=[])

In [ ]:
# Compute the model
sol = gp.compute_model(geo_model, compute_mesh=False)
## This will cause an error, and that is on purpose ##

One *orientation* value is enough two model two layers **given** they are in the same [Series](https://gempy.readthedocs.io/Data/data.Series.html#data.Series). Are they in different Series, this will not work.

The reason is, that GemPy thinks that units in different Series have a different scalar field (or gradient), and are not sub-parallel. Hence, each Series needs its own orientation values in addition to the _Interface_ points to have a working model.

For this model with 2 Series, we thus need **at least** 1 _Orientation_ value (the more the better).

<a id='2.3'></a>
### Again a model with 2 layers and 2 Series
#### ... now with enough input data:

In [ ]:
geo_model = gp.create_model('2layers_2series')

gp.init_data(geo_model, [0., 10., 0., 2., 0., 10.], [100, 3, 100],
            path_i = 'data/02_2Layer_simple_model_interfaces.csv',
            path_o = 'data/02_2Layer_simple_model_2orientation.csv');

In [ ]:
geo_model.surfaces

**Attention** Here, both units belong to the same Series: `Default series`

In [ ]:
gp._plot.plot_data(geo_model)

In [ ]:
# Create the theano model
gp.set_interpolator(geo_model,
                         compile_theano=True,
                         theano_optimizer='fast_compile',
                         verbose=[])

In [ ]:
# Compute the model
sol = gp.compute_model(geo_model, compute_mesh=False)

In [ ]:
gp._plot.plot_section(geo_model, cell_number=2, direction='y', show_data=True)

Adding a second _Orientation_ value changes the model drastically compared to the previous one. The two different orienation values create a gradient-field which curves the layers although the interface points are at the same depth. In addition, the blue unit is curved more than the red one, as the flat orientation value at Z=6 flattens the gradient field. At Z=3, the blue orientation value causes the more inclined gradient field:

In [ ]:
# Darstellen des Gradientenfeldes
gp._plot.plot_scalar_field(geo_model, cell_number=2, direction='y', show_data=True)

<a id='4'></a>
## Fault relations 
Until now, we only looked at continuous rock units. Now we introduce another common feature of geological models: Faults. Faults are discontinuities, which displace rock units. Interfaces and Orientations can be sampled from a fault similarly to a rock unit

Therefore, faults are also modeled similar to geological units in GemPy. They have their own gradient-field, which disturbs the gradient field of geological units. This disturbance causes the displacement of the model surfaces, as those **always** are equal to an iso-line of the gradient-field belonging to the geological unit.

In [ ]:
# Modell with 2 faults
geo_model = gp.create_model('2layers_2faults')

gp.init_data(geo_model, [0., 10., 0., 2., 0., 10.], [100, 3, 100],
            path_i = 'data/04_2Layers_2Faults_interfaces.csv',
            path_o = 'data/04_2Layers_2Faults_orientations.csv');

In [ ]:
geo_model.surfaces

In [ ]:
gp._plot.plot_data(geo_model)

A peculiar thing with faults in Gempy is that they should always have their own Series.

The youngest fault-series is at the top, as is the youngest geological unit in a pile. Generally, in a pile, first come the fault series, then the geological series with the relative position **young** --> **old**.

In [ ]:
# Add faults to series
gp.map_series_to_surfaces(geo_model,
                         {"Fault2_series":'fault2',
                          "Fault1_series":'fault1',
                          "Strati_series":('unit_2', 'unit_1')})

Faults have to be set as faults for gempy (so that their gradient field disturbs the others), and can be assigned the same color:

In [ ]:
# Störungsfarbe
geo_model.set_is_fault(['Fault1_series', 'Fault2_series'], change_color=True)

In [ ]:
# Create the theano model
gp.set_interpolator(geo_model,
                         compile_theano=True,
                         theano_optimizer='fast_compile',
                         verbose=[])

In [ ]:
# Compute the model
sol = gp.compute_model(geo_model)

Plot the result:

In [ ]:
gp._plot.plot_section(geo_model, cell_number=2, direction='y')

In this structure, we see the displacement of blue and purple units along the faults. At the lower model boundary, where the faults cross, we see that the blue unit seems to re-appear. This isn't realistic, probably one fault stops at the other one?

If a fault stops at another one, and also which faults affect which geological units can be defined in a boolean matrix, the `fault_relations_df`:

In [ ]:
# fault relations dataframe
geo_model.faults.faults_relations_df

This fault relation boolean matrix shows True if a fault displaces another series (regardless if it is a geological unit or another fault). Default is: all faults displace all geological units. If we would like to let `Fault1` be affected by `Fault2`, we set the respective entry to `True`.
The `fault_relations_df` is an "upper triangular matrix", so we only look at the upper right half.

In [ ]:
# New Boolean Array
fr = np.array([[False, True, True, True],
               [False, False, True, True],
               [False, False, False, False],
               [False, False, False, False]])

In [ ]:
geo_model._faults.set_fault_relation(fr)

In [ ]:
# Create the theano model
gp.set_interpolator(geo_model,
                         compile_theano=True,
                         theano_optimizer='fast_compile',
                         verbose=[])

In [ ]:
# Compute the model
sol = gp.compute_model(geo_model, compute_mesh=False)

In [ ]:
# Show the faults
gp._plot.plot_section(geo_model, cell_number=1, direction='y', show_data=True)

Here we see that `Fault1` stopps at the other fault, and follows now `Fault2`. Faults are visible throughout the model space, but don't affect the whole model domain!

<a id='5'></a>
## Onlap Relations

Series in GemPy can have two different _Relations_ compared to older, underlying Series. The two _Relations_ are `Onlap` or `Erode`. Per default all younger units erode older units, so their bottom_relation is `Erode`. If this is changed to `Onlap`, a younger Unit stops at an older one.

Let's test this in a model:

In [ ]:
# Model creation
geo_model = gp.create_model('Test_model')

In [ ]:
gp.init_data(geo_model, [0, 10., 0, 2., 0, 5.], [100, 3, 100],
            path_o = 'data/05_toy_fold_unconformity_orientations.csv',
             path_i = 'data/05_toy_fold_unconformity_interfaces.csv', default_values=True);

In [ ]:
geo_model.surfaces

In [ ]:
# Map the units to series
gp.map_series_to_surfaces(geo_model,
                         {"Flat_Series":'Flat',
                          "Inclined_Series":'Inclined',
                          "Fold_Series": ('Basefold', 'Topfold')})

Now, all units have their own Series. A "folded" one in the lower left corner, a single "inclined" Series and a flat one, the youngest:

In [ ]:
gp._plot.plot_data(geo_model, direction='y')

<div class="alert alert-info"> 
    
**Question:** 
All Units have `BottomRelation` `Erode` gesetzt. How do you think the resulting model will look like?
</div>

In [ ]:
geo_model.series

In [ ]:
# Create the theano model
gp.set_interpolator(geo_model,
                         compile_theano=True,
                         theano_optimizer='fast_compile',
                         verbose=[])

In [ ]:
# Compute the model
sol = gp.compute_model(geo_model, compute_mesh=False)

In [ ]:
gp._plot.plot_section(geo_model, cell_number=2, direction='y')

As expected?

<a id='5.1'></a>
### Inclined as ONLAP

Now let's have a look what happens if `Inclined` is set to `BottomRelation` Onlap.

In [ ]:
# Inclined to Onlap
geo_model._series.set_bottom_relation('Inclined_Series', bottom_relation='Onlap')

In [ ]:
# Data Plot
gp._plot.plot_data(geo_model, direction='y')

In [ ]:
# Create the theano model
gp.set_interpolator(geo_model,
                         compile_theano=True,
                         theano_optimizer='fast_compile',
                         verbose=[])
# Compute the model
sol = gp.compute_model(geo_model, compute_mesh=False)

In [ ]:
# result Plot
gp._plot.plot_section(geo_model, cell_number=2, direction='y')

This might be somewhat confusing? But when we look closely, this actually makes Sense!

<a id='5.2'></a>
### Inclined and Flat as ONLAP

Now next to `Inclined` we also set the Series `Flat` on _Onlap_, and have a look at the result:

In [ ]:
# both young series are now onlap:
geo_model._series.set_bottom_relation(['Flat_Series','Inclined_Series'], bottom_relation='Onlap')

In [ ]:
# Create the theano model
gp.set_interpolator(geo_model,
                         compile_theano=True,
                         theano_optimizer='fast_compile',
                         verbose=[])
# Compute the model
sol = gp.compute_model(geo_model, compute_mesh=False)

In [ ]:
# result plot
gp._plot.plot_section(geo_model, cell_number=2, direction='y')

Let's have a look at the model in 3D

In [ ]:
gp.plot_3d(geo_model, plotter_type='background')

<a id='6'></a>
## Final thoughts

In this notebook, we tried to give an overview over the most basic GemPy functionalities, Surfaces, Series, Relations...all that builds the basis of GemPy, but its functionality goes way beyond that. However, it is important to understand the basics shown here, as they help to build more complex models (with a robust fundament).

In [ ]:
print("That's all folks!")